# Read or Write GAMMA files

**Copyright (c) 2021, Lei Yuan**

## all functions

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
from pykml.factory import KML_ElementMaker as KML
import matplotlib.pyplot as plt
import matplotlib as mpl
from lxml import etree
import numpy as np
import os
import sys
import zipfile
from osgeo import gdal, osr
from matplotlib.colors import LightSource, ListedColormap
from matplotlib.cm import register_cmap
%matplotlib inline


def read_gamma(file, lines, file_type):
    # check file
    if not os.path.isfile(file):
        sys.exit('{} does not exist.'.format(file))
    data = np.fromfile(file, dtype=file_type)
    # GAMMA output files are big-endian
    data.byteswap('True')
    data = data.reshape(lines, -1)

    return data


def write_gamma(data, out_file, file_type):
    data = data.astype(file_type)
    data.byteswap('True')
    data.reshape(-1, 1)
    print('Writing data to {}'.format(out_file))
    data.tofile(out_file)
    print('done.')


def plot_data(data, cmap='jet', title=None, figsize=(10, 10)):
    fig, ax = plt.subplots(figsize=figsize)
    if title:
        ax.set_title(title)
    im = ax.imshow(data, cmap=cmap)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes('right', size='6%', pad=0.2)
    fig.colorbar(im, cax=cax)


def wrap_data(data, wrap_range=[-np.pi, np.pi]):
    w0, w1 = wrap_range
    wrapped_data = w0 + np.mod(data - w0, w1 - w0)

    return wrapped_data


def read_dem_par(par_file):
    par_key = [
        'width', 'nlines', 'corner_lat', 'corner_lon', 'post_lat', 'post_lon'
    ]
    par_value = []
    with open(par_file, 'r') as f:
        for line in f.readlines():
            for i in par_key:
                if line.strip().startswith(i):
                    value = line.strip().split()[1]
                    if i in ['width', 'nlines']:
                        par_value.append(int(value))
                    else:
                        par_value.append(float(value))

    return par_value


def get_lon_lat(dem_par):
    par_value = read_dem_par(dem_par)
    north = par_value[2]
    south = north + par_value[4] * par_value[1]
    west = par_value[3]
    east = west + par_value[5] * par_value[0]

    return [south, north, west, east]


def data2xyz(data, dem_par, out_file):
    par_value = read_dem_par(dem_par)
    width, length = par_value[0], par_value[1]
    upper_left_lat, upper_left_lon = par_value[2], par_value[3]
    lat_step, lon_step = par_value[4], par_value[5]

    if data.shape[1] == width and data.shape[0] == length:
        lon = np.linspace(upper_left_lon, upper_left_lon + lon_step * width, width)
        lat = np.linspace(upper_left_lat, upper_left_lat + lat_step * length, length)
        lons, lats = np.meshgrid(lon, lat)

        lons = lons.reshape((-1, 1))
        lats = lats.reshape((-1, 1))
        data = data.reshape((-1, 1))

        not_nan = ~np.isnan(data)
        data = data[not_nan].reshape((-1, 1))
        lons = lons[not_nan].reshape((-1, 1))
        lats = lats[not_nan].reshape((-1, 1))

        out_data = np.hstack((lons, lats, data))
        print('Writing data to {}'.format(out_file))
        np.savetxt(out_file, out_data, fmt='%4f')
        print('done.')
    else:
        print('Error data.')


def data2geotiff(data, dem_par, outfile):
    par_value = read_dem_par(dem_par)
    width, nlines = par_value[0], par_value[1]
    lat_step, lon_step = par_value[4], par_value[5]
    lon_lat = get_lon_lat(dem_par)
    max_lat, min_lon = lon_lat[1], lon_lat[2]

    driver = gdal.GetDriverByName('GTiff')
    dataset = driver.Create(outfile, width, nlines, 1, gdal.GDT_Float32)

    dataset.SetGeoTransform([min_lon, lon_step, 0, max_lat, 0, lat_step])

    sr = osr.SpatialReference()
    sr.SetWellKnownGeogCS('WGS84')

    dataset.SetProjection(sr.ExportToWkt())
    dataset.GetRasterBand(1).WriteArray(data)
    print('Writing data to {}.'.format(outfile))
    dataset.FlushCache()
    dataset = None
    print('done.')


def plot_colorbar(vlim, cmap, figsize, nbins, alpha, out_file):
    fig, cax = plt.subplots(figsize=figsize)
    norm = mpl.colors.Normalize(vmin=vlim[0], vmax=vlim[1])
    cmap = plt.get_cmap(cmap)
    cbar = mpl.colorbar.ColorbarBase(cax, cmap=cmap, norm=norm, orientation='vertical')
    cbar.locator = mpl.ticker.MaxNLocator(nbins=nbins)
    cbar.update_ticks()
    cbar.ax.tick_params(which='both', labelsize=12)
    fig.patch.set_facecolor('white')
    fig.patch.set_alpha(alpha)
    fig.savefig(out_file, bbox_inches='tight', facecolor=fig.get_facecolor(), dpi=300)


def plot_img(data, vlim, cmap, alpha, dpi, out_file, dem_data=None):
    fig = plt.figure(frameon=False)
    ax = fig.add_axes([0., 0., 1., 1.])
    ax.set_axis_off()
    cmap = plt.get_cmap(cmap)
    if dem_data is None:
        ax.imshow(data, aspect='auto', vmax=vlim[1], vmin=vlim[0], cmap=cmap)
    else:
        ls = LightSource(azdeg=315, altdeg=45)
        dem_shade = ls.shade(dem_data, cmap=plt.get_cmap('gray'))
        ax.imshow(dem_shade, zorder=0)
        ax.imshow(data, aspect='auto', vmax=vlim[1], vmin=vlim[0], cmap=cmap, zorder=1, alpha=alpha)
    fig.savefig(out_file, pad_inches=0.0, transparent=True, interpolation='nearest', dpi=dpi)


def write_kmz(data, lon_lat, out_file, dem_data=None, cmap='jet', alpha=1, dpi=300, colorbar_size=(0.18, 3.6), nbins=2, vlim=[-60, 60]):
    out_file = os.path.abspath(out_file)
    if out_file.endswith('.kmz'):
        out_file = out_file[:-4]

    out_dir = os.path.dirname(out_file)

    colorbar_file = os.path.join(out_dir, 'colorbar.png')
    plot_colorbar(vlim, cmap, colorbar_size, nbins, alpha, colorbar_file)

    img_file = os.path.join(out_dir, 'img.png')
    plot_img(data, vlim, cmap, alpha, dpi, img_file, dem_data=dem_data)

    south, north, west, east = lon_lat

    doc = KML.kml(KML.Folder(KML.name(os.path.basename(out_file))))
    img_displayed = KML.GroundOverlay(
        KML.name('img'), KML.Icon(KML.href('img.png')),
        KML.LatLonBox(KML.north(str(north)), KML.south(str(south)),
                      KML.east(str(east)), KML.west(str(west))))
    doc.Folder.append(img_displayed)

    legend = KML.ScreenOverlay(
        KML.name('colorbar'),
        KML.Icon(KML.href('colorbar.png'), KML.viewBoundScale(0.75)),
        KML.overlayXY(
            x="0.0",
            y="1",
            xunits="fraction",
            yunits="fraction",
        ), KML.screenXY(
            x="0.0",
            y="1",
            xunits="fraction",
            yunits="fraction",
        ),
        KML.rotationXY(
            x="0.",
            y="1.",
            xunits="fraction",
            yunits="fraction",
        ), KML.size(
            x="0",
            y="250",
            xunits="pixel",
            yunits="pixel",
        ), KML.visibility(1), KML.open(0))

    doc.Folder.append(legend)

    kml_str = etree.tostring(doc, pretty_print=True)
    kml_name = out_file + '.kml'
    with open(kml_name, 'wb') as f:
        f.write(kml_str)

    kmz_name = out_file + '.kmz'
    with zipfile.ZipFile(kmz_name, 'w') as f:
        os.chdir(out_dir)
        f.write(os.path.basename(kml_name))
        os.remove(os.path.basename(kml_name))
        f.write('img.png')
        os.remove('img.png')
        f.write('colorbar.png')
        os.remove('colorbar.png')


def register_cmap(cmap_file, cmap_name):
    cm = ListedColormap(
        np.loadtxt(cmap_file, dtype='float') / 255.0, cmap_name)
    plt.cm.register_cmap(cmap=cm)

    return cm


cm = register_cmap(os.path.join(os.environ['TINTPY_HOME'], 'tintpy', 'data', 'colormaps', 'rmg_hs.cm'), 'rmg_hs')

## read unwrapped file

In [ ]:
os.chdir(r'/ly/diff_2d')

unw = read_gamma('unw', 5848, np.float32)

plot_data(unw, cmap='rmg_hs', title='unw', figsize=(8, 8))

# write_gamma(unw, 'file_by_python.unw', np.float32)

## read coherence

In [ ]:
cc = read_gamma('20171119_20171213.cc', 465, 'float32')
cc[cc == 0] = np.nan

plot_data(cc, cmap='gray', title='cc')

# write_gamma(cc, 'file_by_python.cc', 'float32)

## read interferogram

In [ ]:
data = read_gamma('20171119_20171225.adf.diff', 465, 'complex64')

# rmg_hs is recommanded for interferogram
plot_data(np.angle(data), cmap='rmg_hs', title='diff')

# write_gamma(data, 'file_by_python.diff', 'complex64)

## wrap data

In [ ]:
data = read_gamma('20171119_20171225.adf.unw', 465, 'float32')

wrapped_data = wrap_data(data, wrap_range=[-np.pi, np.pi])

# rmg_hs is recommanded for interferogram
plot_data(wrapped_data, cmap='rmg_hs', title='wrap')

## mask data using coherence

In [ ]:
unw = read_gamma('20171119_20171225.adf.unw', 465, 'float32')
cc = read_gamma('20171119_20171213.cc', 465, 'float32')

unw[cc < 0.2] = np.nan

plot_data(unw, cmap='jet', title='masked unw')

## data2xyz

In [ ]:
unw = read_gamma('20171119_20171225.adf.unw', 465, 'float32')

data2xyz(unw, 'dem_seg.par', 'unw.txt')

## data2geotiff

In [ ]:
unw = read_gamma('20171119_20171225.adf.unw', 465, 'float32')

data2geotiff(unw, 'dem_seg.par', 'unw.tif')

## data2kmz

In [ ]:
lon_lat = get_lon_lat('dem_seg.par')
_, nlines, _, _, _, _ = read_dem_par('dem_seg.par')

unw = read_gamma('geo_ph_rate', nlines, 'float32')
unw[unw == 0] = np.nan

write_kmz(unw, lon_lat, 'wrap.kmz', cmap='rmg_hs', value_range=[-10, 10], alpha=1)

In [ ]:
# using dem as background image
lon_lat = get_lon_lat('dem_seg.par')
_, nlines, _, _, _, _ = read_dem_par('dem_seg.par')

unw = read_gamma('geo_ph_rate', nlines, 'float32')

dem_data = read_gamma('dem_seg', nlines, 'float32')

write_kmz(unw, lon_lat, 'unw.kmz', cmap='jet', vlim=[-10, 10], alpha=0.6, dem_data=dem_data, dpi=500)